In [36]:
import findspark
import os

findspark.init(os.environ.get('SPARK_HOME'))

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as functions
from decimal import Decimal


In [37]:
spark = SparkSession.builder.appName("GastoMinisterio").getOrCreate()

In [38]:
df = spark.read.format("csv").option("header", True).option("delimiter", ";").option('encoding', 'windows-1252').csv('hdfs://localhost:9000/user/maycon/2019_Viagem.csv')   

In [39]:
to_value = lambda v : Decimal(v.replace(",", "."))

In [40]:
udf_to_value = functions.udf(to_value)

In [41]:
df2 = df.withColumn("value", udf_to_value(df["Valor passagens"]))

In [42]:
orgaos = df2.select("Nome do órgão superior").distinct()

In [43]:
df2.groupBy("Nome do órgão superior").agg(functions.max("value"), functions.sum("value"), functions.count("value"), functions.avg("value")).show(orgaos.count(), truncate = False)

+-----------------------------------------------------------+----------+--------------------+------------+------------------+
|Nome do órgão superior                                     |max(value)|sum(value)          |count(value)|avg(value)        |
+-----------------------------------------------------------+----------+--------------------+------------+------------------+
|Ministério do Meio Ambiente                                |989.92    |4754103.779999999   |11766       |404.0543753187149 |
|Ministério da Agricultura, Pecuária e Abastecimento        |998.83    |6569691.700000001   |18862       |348.30302725055674|
|Advocacia-Geral da União                                   |994.57    |978350.1899999995   |3696        |264.70513798701285|
|Ministério das Cidades                                     |0.00      |0.0                 |1           |0.0               |
|Ministério da Cultura                                      |3494.56   |5256.2              |2           |2628.1      

In [48]:
df2.write.mode('overwrite').csv("hdfs://localhost:9000/user/maycon/agrupamentoMinisterio2.csv")

In [60]:
df2.select('Nome do órgão superior', 'value').show(truncate = False)

+-----------------------------------------+-------+
|Nome do órgão superior                   |value  |
+-----------------------------------------+-------+
|Ministério da Educação                   |2875.92|
|Ministério da Educação                   |2420.48|
|Ministério da Educação                   |2694.58|
|Ministério da Defesa                     |1236.38|
|Ministério da Defesa                     |746.35 |
|Ministério da Defesa                     |1293.41|
|Ministério da Educação                   |986.15 |
|Ministério da Justiça e Segurança Pública|1492.03|
|Ministério da Justiça e Segurança Pública|1492.03|
|Ministério da Educação                   |672.60 |
|Ministério da Educação                   |2671.65|
|Ministério da Educação                   |0.00   |
|Ministério das Relações Exteriores       |3463.12|
|Ministério das Relações Exteriores       |5895.37|
|Ministério da Educação                   |5889.17|
|Ministério da Justiça e Segurança Pública|4304.31|
|Ministério 

In [63]:
tuplaOrgaosValor = df2.rdd.map(lambda x : (x["Nome do órgão superior"]), x.[getAs[Decimal](]"value")))

In [64]:
tuplaOrgaosValor.take(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 47.0 failed 1 times, most recent failure: Lost task 0.0 in stage 47.0 (TID 1467, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/maycon/programas/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/sql/types.py", line 1527, in __getattr__
    idx = self.__fields__.index(item)
ValueError: 'getAs' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/maycon/programas/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/maycon/programas/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  Fil